In [17]:
%%bigquery 
--Count storm events vs fires per state


    
    
create view reporting.v_Storm_and_fires_per_states as
SELECT h.state, Event_Amount as Storms , fires as fires
FROM
(SELECT state, COUNT(event_type) AS Event_Amount
FROM `trusty-wavelet-252622.storm_events_modeled.Curated_StormEvents_Beam` c
JOIN  (SELECT a.county_id, a.name, b.state 
FROM `trusty-wavelet-252622.storm_events_modeled.Locations_Beam` a
JOIN  `trusty-wavelet-252622.storm_events_modeled.State` b 
on a.state_fips = b.state_fips) d
on c.county_id = d.county_id
GROUP BY state) h 
JOIN(SELECT state, COUNT(incident_id) AS fires
FROM ( SELECT incident_id, c.state as state
FROM `trusty-wavelet-252622.WildLand_Incidents_modeled.Incidents_Beam` a JOIN 
(SELECT b.statename as state, b.state_id
FROM `trusty-wavelet-252622.WildLand_Incidents_modeled.State_Beam` b) c
on a.state = c.state_id) 
GROUP BY state) g
on h.state = g.state
order BY h.state

""


In [18]:
%%bigquery 

--Count storm events vs fires per month

create view reporting.v_Storm_and_fires_per_month as
SELECT a.month, a.Event_Amount as Storms, b.Event_Amount as Fires
FROM (SELECT month_name as month , COUNT(event_type) AS Event_Amount
FROM `trusty-wavelet-252622.storm_events_modeled.Curated_StormEvents_Beam` c
where c.year = 2011
group by month)  a
JOIN 
(SELECT month, COUNT(event_id) AS Event_Amount
FROM (select a.incident_id as event_id, b.monthname as month
FROM `trusty-wavelet-252622.WildLand_Incidents_modeled.Incidents_Beam` a
join (select date, monthname
FROM `trusty-wavelet-252622.WildLand_Incidents_modeled.Date_Beam`
where year = 2011) b
on a.date = b.date)
group by month) b
ON a.month = b.month




""


In [19]:
%%bigquery 

-- Crops burned per months by storm and fires

create view reporting.v_crops_burned_per_storm_and_fires as

SELECT r.month, average_damaged_crops as crops_by_storm, crops_burned as crops_by_fires
FROM (SELECT month_name , count(damage_crops) average_damaged_crops
FROM `trusty-wavelet-252622.storm_events_modeled.Curated_StormEvents_Beam` a
where a.year = 2010 or a.year = 2011
Group by month_name
ORDER BY average_damaged_crops) t


JOIN

(SELECT month, COUNT( crop_burn1 ) AS crops_burned
FROM (select a.incident_id as crop_burn1 , b.monthname as month
FROM `trusty-wavelet-252622.WildLand_Incidents_modeled.Incidents_Beam` a
join (select date, monthname
FROM `trusty-wavelet-252622.WildLand_Incidents_modeled.Date_Beam`
where year = 2011) b 
on a.date = b.date) group by month) r
on t.month_name = r.month

""
